In [1]:
import time
import os 
from dotenv import load_dotenv
from get_history import history
load_dotenv()

def response_text(openai_resp):
    return openai_resp['choices'][0]['message']['content']

from gptcache import cache
from gptcache.adapter import openai
from gptcache.embedding import Onnx
from gptcache.manager import CacheBase, VectorBase, get_data_manager
from gptcache.similarity_evaluation.distance import SearchDistanceEvaluation

print("Cache loading.....")

onnx = Onnx()
data_manager = get_data_manager(CacheBase("sqlite"), VectorBase("faiss", dimension=onnx.dimension),data_path="./sqlite.db")
cache.init(
    embedding_func=onnx.to_embeddings,
    data_manager=data_manager,
    similarity_evaluation=SearchDistanceEvaluation(),
    )
# cache.import_data(questions=["my phone number is 4158887963."],answers=["you phone number is 4158887963,correct?"])
cache.set_openai_key()

conversation_history = history

while True:
    # Get user input from the keyboard
    question = input("Enter your question (or type 'q' to quit): ")
    
    # Exit the loop if the user types 'exit'
    if question.lower() == 'q':
        break
    
    start_time = time.time()
    
    # Append the new question to the conversation history
    conversation_history.append({"role": "user", "content": question})
    
    response = openai.ChatCompletion.create(
        model='gpt-4o',
        messages=conversation_history  # Use the accumulated conversation history
    )
    # Append the response to the conversation history
    conversation_history.append({"role": "assistant", "content": response_text(response)})
    
    print(f'Question: {question}')
    if "gptcache" in response.keys():
        print("Cache hit Time consuming: {:.2f}s".format(time.time() - start_time))
    else:
        print("Cache not hit Time consuming: {:.2f}s".format(time.time() - start_time))
    print(f'Answer: {response_text(response)}\\n')

/Users/zxs/miniconda3/envs/llm_cache/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


Cache loading.....
Question: hello
Cache not hit Time consuming: 4.02s
Answer: hi there! how can i assist you with your order today at happy grill?\n
Question: hello
Cache hit Time consuming: 0.27s
Answer: hi there! how can i assist you with your order today at happy grill?\n
